Evaluation in µDhall is based on applying just three rules:

- Evaluate all function applications by substituting variables.
- Evaluate built-in operations and functions (for example, `10 + 20` or `Natural/subtract 1 2`) according to their specific meaning.
- If the result still contains any lambdas, rename all bound variables to `_` and introduce enough de Bruijn indices to disambiguate name clashes.

Evaluation will take a µDhall expression and produce an equivalent expression in the "normal form". The normal form cannot be evaluated any further. In most cases, the normal form will be a simple value (say, a `Natural` number).

For example, `(λ(y : Natural) → x + 10) 123` is evaluated to the normal form `133`.

However, in some cases the normal form is not a simple value but a function; and it can even be longer than the initial expression.
This is because evaluation in µDhall is _symbolic_. In µDhall, functions are applied even under lambda where some operands are variable symbols.

For example, this expression:

     λ(x : Natural) → (λ(y : Natural) → x + y) 123

is evaluated to the normal form `λ(_ : Natural) → _ + 123`. We have simplified `(λ(y : Natural) → x + y) 123` to `x + 123`, while `x` remains a variable symbol. Then we renamed `x` to `_`, replacing `λ(x : Natural) → x + 123` by `λ(_ : Natural) → _ + 123`.

When bound variables are renamed to `_`, we will introduce de Bruijn indices whenever we have to avoid a name clash. For example:

     λ(a : Type) → λ(b : Type) → a

is renamed to:

     λ(_ : Type) → λ(_ : Type) → _@1

If the variable `_` is already present, we may also need to increment its de Bruijn index:

     λ(x : Type) → _

will become:

     λ(_ : Type) → _@1


Evaluating by substition is called **beta-normalization**. Renaming bound variables is called **alpha-normalization**.

We specify the formal rules for these normalization operations using the "proof notation". We denote by `a ↦ b` a proof that the alpha-normalization of `a` will give `b`, and by `a ⇥ b` a proof that the beta-normalization of `a` will give `b`.

For example, the beta-normalization rules for adding natural numbers look like this:

```
l ⇥ NaturalLiteral(0)   r₀ ⇥ r₁
───────────────────────────────
l + r₀ ⇥ r₁


l₀ ⇥ l₁   r ⇥ NaturalLiteral(0)
───────────────────────────────
l₀ + r ⇥ l₁


l ⇥ NaturalLiteral(m)   r ⇥ NaturalLiteral(m)
─────────────────────────────────────────────  ; Here "m + n" means "use built-in addition of integers".
l + r ⇥ NaturalLiteral(m + n)


l₀ ⇥ l₁   r₀ ⇥ r₁
─────────────────  ; If no other rule matches.
l₀ + r₀ ⇥ l₁ + r₁
```

These four rules show how to compute the beta-normalization for any expression of the form `a + b`. We will need to specify how these operations work with all possible expression structures. This will allow us to translate the specification directly into code.